# 🧱 Building Scalable Python Applications: Modules and Packages

**Welcome!** As Python projects grow, organizing code becomes paramount. This notebook delves into Python's fundamental organizational structures: **Modules** and **Packages**. Understanding how to create, import, and structure them is essential for writing maintainable, reusable, and scalable code.

**Target Audience:** Python developers moving beyond single-file scripts, aiming to build larger, well-structured applications.

**Learning Objectives:**
*   Understand what constitutes a Python module and how to create one.
*   Master various import statements (`import`, `from ... import`, aliasing).
*   Learn how Python finds modules (`sys.path`).
*   Understand package structure and the role of `__init__.py`.
*   Differentiate between and use absolute and relative imports.
*   Explore modern packaging concepts (namespace packages, `pyproject.toml`).
*   Learn best practices for organizing projects and managing dependencies.
*   Identify common pitfalls like circular imports and namespace pollution.

## 1. Introduction: Why Organize Code?

Imagine writing a large book as one single, massive paragraph. It would be impossible to read, navigate, or edit. Similarly, writing large software applications in a single file becomes unmanageable quickly.

Python's modules and packages provide mechanisms to break down code into logical, reusable units, offering several key benefits:

*   **Organization:** Group related code (functions, classes, variables) together, making the project structure clear and understandable.
*   **Reusability:** Write code once in a module and import it wherever needed, avoiding repetition (DRY - Don't Repeat Yourself).
*   **Maintainability:** Changes made within one module are less likely to break unrelated parts of the application (if designed well).
*   **Namespace Management:** Avoid naming conflicts between different parts of your code. Each module has its own private namespace.
*   **Collaboration:** Different developers can work on different modules/packages simultaneously with fewer conflicts.

**Analogy: The Toolbox and Workshop**

*   A **Module** is like a specialized tool or a small set of related tools (e.g., a screwdriver set - `screwdrivers.py`) stored in a file.
*   A **Package** is like a drawer or a section in your workshop (a directory) that holds multiple toolsets (modules) and possibly smaller organizers (subpackages), all related to a specific task (e.g., a `woodworking` package containing `chisels.py`, `saws.py`, and a `fasteners` subpackage).
*   The `__init__.py` file acts like the label on the drawer, identifying it as a package and potentially listing the main tools available directly from the drawer label.
*   **Importing** is like taking a specific tool (`import screwdrivers`) or a specific type of screwdriver (`from screwdrivers import phillips_head`) out of the toolbox to use it.

## 2. Modules: The Basic Unit of Code Organization

**What is a Module?**
Simply put, any file ending in `.py` containing Python definitions and statements can be considered a module. The filename becomes the module name (without the `.py` extension).

### 2.1 Creating a Simple Module

Let's imagine we create a file named `string_utils.py` with some helper functions:

```python
# string_utils.py (Imagine this is saved as a separate file)
"""A module with utility functions for strings."""

import logging

# Configure logging for the module
logger = logging.getLogger(__name__) # Best practice: use module name
logger.addHandler(logging.NullHandler()) # Avoid adding handlers here by default

__version__ = "0.1.0" # Module version (good practice)

def reverse_string(s: str) -> str:
    """Reverses a given string."""
    logger.debug(f"Reversing string: '{s}'")
    return s[::-1]

def count_vowels(s: str) -> int:
    """Counts the number of vowels (a, e, i, o, u) in a string."""
    vowels = "aeiouAEIOU"
    count = sum(1 for char in s if char in vowels)
    logger.debug(f"Counted {count} vowels in '{s}'")
    return count

# The __name__ == '__main__' block
# Code inside this block only runs when the file is executed directly
# as a script, NOT when it's imported as a module.
if __name__ == '__main__':
    # Example usage or simple tests when run directly
    logging.basicConfig(level=logging.DEBUG) # Configure logging for direct run
    logger.info("string_utils.py executed directly.")
    test_string = "Hello World"
    print(f"Original: '{test_string}'")
    print(f"Reversed: '{reverse_string(test_string)}'")
    print(f"Vowel Count: {count_vowels(test_string)}")
```

### 2.2 Importing Modules

To use the functions defined in `string_utils.py` in another script (or this notebook), you need to `import` it. **Crucially, the module file (`string_utils.py`) must be located somewhere Python can find it.**

#### How Python Finds Modules: `sys.path`

When you execute `import my_module`, Python searches for `my_module.py` (or a package named `my_module`) in the following locations, in order:

1.  **The directory containing the input script** (or the current directory if running interactively).
2.  Directories listed in the **`PYTHONPATH`** environment variable (if set).
3.  Installation-dependent default paths (usually includes the `site-packages` directory where third-party libraries are installed).

You can inspect the search path using `sys.path`.

In [1]:
import sys

print("--- Python Module Search Path (sys.path) ---")
for i, path in enumerate(sys.path):
    print(f"{i}: {path}")

--- Python Module Search Path (sys.path) ---
0: /usr/lib/python313.zip
1: /usr/lib/python3.13
2: /usr/lib/python3.13/lib-dynload
3: 
4: /usr/lib/python3.13/site-packages


#### Import Statements

**(Note:** For the following code cells to work, you would need to actually create the `string_utils.py` file in the same directory as this notebook, or in a directory listed in `sys.path`.)

In [2]:
# Assume string_utils.py exists in the current directory or sys.path
import logging

# Option 1: Import the entire module
print("\n--- Importing module: import string_utils ---")
try:
    import string_utils 
    
    # Access functions using module_name.function_name
    reversed_text = string_utils.reverse_string("Python Module")
    vowel_count = string_utils.count_vowels("Python Module")
    print(f"Module Version: {string_utils.__version__}")
    print(f"Reversed: '{reversed_text}'")
    print(f"Vowels: {vowel_count}")

    # Configure logging for the imported module from the main script
    # This allows the main application to control log levels/handlers
    logging.getLogger('string_utils').setLevel(logging.DEBUG)
    logging.getLogger('string_utils').addHandler(logging.StreamHandler())
    print("Running count_vowels again with DEBUG logging enabled:")
    string_utils.count_vowels("Debug Me") 
    
except ModuleNotFoundError:
    print("ERROR: string_utils.py not found. Please create the file.")
except AttributeError as e:
     print(f"ERROR: Problem accessing module attribute: {e}")


--- Importing module: import string_utils ---
ERROR: string_utils.py not found. Please create the file.


In [3]:
# Option 2: Import specific names from the module
print("\n--- Importing specific names: from string_utils import ... ---")
try:
    from string_utils import reverse_string, count_vowels
    
    # Access functions directly by name
    reversed_direct = reverse_string("Direct Import")
    vowels_direct = count_vowels("Direct Import")
    print(f"Reversed (direct): '{reversed_direct}'")
    print(f"Vowels (direct): {vowels_direct}")
    
    # Note: Module version __version__ is not directly imported here
    # print(string_utils.__version__) # Would cause NameError unless string_utils also imported

except ModuleNotFoundError:
    print("ERROR: string_utils.py not found. Please create the file.")
except ImportError as e:
     print(f"ERROR: Could not import specific names: {e}")


--- Importing specific names: from string_utils import ... ---
ERROR: string_utils.py not found. Please create the file.


In [4]:
# Option 3: Import using an alias
print("\n--- Importing with alias: import string_utils as su ---")
try:
    import string_utils as su # Common convention for longer names
    
    reversed_alias = su.reverse_string("Aliased")
    vowels_alias = su.count_vowels("Aliased")
    print(f"Version (alias): {su.__version__}")
    print(f"Reversed (alias): '{reversed_alias}'")
    print(f"Vowels (alias): {vowels_alias}")
    
except ModuleNotFoundError:
    print("ERROR: string_utils.py not found. Please create the file.")


--- Importing with alias: import string_utils as su ---
ERROR: string_utils.py not found. Please create the file.


In [5]:
# Option 4: Import specific names with aliases
print("\n--- Importing specific names with aliases ---")
try:
    from string_utils import reverse_string as rev_str, count_vowels as count_v
    
    reversed_named_alias = rev_str("Specific Alias")
    vowels_named_alias = count_v("Specific Alias")
    print(f"Reversed (named alias): '{reversed_named_alias}'")
    print(f"Vowels (named alias): {vowels_named_alias}")
    
except ModuleNotFoundError:
    print("ERROR: string_utils.py not found. Please create the file.")
except ImportError as e:
     print(f"ERROR: Could not import specific names: {e}")


--- Importing specific names with aliases ---
ERROR: string_utils.py not found. Please create the file.


In [6]:
# Option 5: Import everything (*) - **Generally Discouraged!**
print("\n--- Importing everything: from string_utils import * (Discouraged) ---")
try:
    # This imports all names NOT starting with an underscore 
    # unless the module defines __all__.
    from string_utils import * 

    # Can lead to namespace pollution and makes it unclear where functions come from.
    reversed_star = reverse_string("Star Import") 
    vowels_star = count_vowels("Star Import")
    print(f"Reversed (star): '{reversed_star}'")
    print(f"Vowels (star): {vowels_star}")
    print(f"Version (star): {__version__}") # Also imports variables like __version__

except ModuleNotFoundError:
    print("ERROR: string_utils.py not found. Please create the file.")
except NameError as e:
    # Might occur if names starting with _ were intended to be imported
     print(f"ERROR: Name error likely due to star import specifics: {e}")


--- Importing everything: from string_utils import * (Discouraged) ---
ERROR: string_utils.py not found. Please create the file.


### 2.3 The `__name__ == '__main__'` Idiom

As seen in `string_utils.py`:

*   When a Python file is run directly (e.g., `python string_utils.py`), the special built-in variable `__name__` for that file is set to the string `'__main__'`. 
*   When the same file is imported as a module into another file, its `__name__` variable is set to the module's actual name (e.g., `'string_utils'`).

This allows you to write code (like tests, examples, or script logic) inside the `if __name__ == '__main__':` block that will only execute when the file is run directly, and not when it's imported elsewhere. This is standard practice for modules that might also be useful as standalone scripts or need simple self-tests.

## 3. Packages: Organizing Modules

**What is a Package?**
A package is a way to structure Python's module namespace using directories. It's essentially a directory containing:
1.  One or more module files (`.py`).
2.  Possibly other sub-packages (subdirectories also containing an `__init__.py`).
3.  An `__init__.py` file (even if empty).

### 3.1 The Role of `__init__.py`

This file has several purposes:

1.  **Marker:** Historically (before Python 3.3), its presence marked a directory as a Python package, distinguishing it from a regular directory that shouldn't be importable.
2.  **Initialization:** Code in `__init__.py` is executed *the first time* the package or any of its modules are imported. This can be used for package-level setup (though complex setup is often better done elsewhere).
3.  **Namespace Control:** It can define what symbols are available when importing the package directly (e.g., `import mypackage`).
4.  **Convenience Imports:** It can import selected functions/classes from its submodules, making them available at the top level of the package (e.g., allowing `from mypackage import useful_function` instead of `from mypackage.submodule import useful_function`).
5.  **`__all__`:** Define a list named `__all__` in `__init__.py` to specify exactly what names should be imported when `from mypackage import *` is used. This overrides the default behavior of importing all names not starting with an underscore.

**Modern Note (Namespace Packages):** Since Python 3.3, directories *without* an `__init__.py` can also be treated as **namespace packages**. These allow splitting a single logical package across multiple directories (useful for large libraries or plugins). However, for standard package creation within a single project, including an `__init__.py` (even if empty) remains common practice and necessary if you want initialization code or to use `__all__`.

### 3.2 Creating a Simple Package

Let's create a hypothetical package structure:

```
my_project/
├── main_script.py
└── mypackage/
    ├── __init__.py
    ├── core_logic.py
    └── utils/
        ├── __init__.py
        └── helpers.py
```

**File Contents (Illustrative):**

```python
# mypackage/__init__.py
"""My awesome package root."""
print("Executing mypackage/__init__.py")

# Option 1: Make specific functions available directly from the package
from .core_logic import process_data 
from .utils.helpers import format_output

# Option 2: Define __all__ to control 'from mypackage import *'
__all__ = ['process_data', 'format_output', 'core_logic', 'utils'] 
# Without __all__, 'from mypackage import *' would only import names defined
# *directly* within this __init__.py, NOT the imported ones above.
# With __all__, it imports exactly what's listed.

PACKAGE_VERSION = "1.0"
```

```python
# mypackage/core_logic.py
"""Core processing functions."""

from .utils import helpers # Example of relative import within the package

def process_data(data):
    print(f"Core logic processing: {data}")
    # Use a helper from the subpackage
    formatted = helpers.format_output(data, prefix="[CORE]")
    return formatted
```

```python
# mypackage/utils/__init__.py
"""Utilities subpackage."""
print("Executing mypackage/utils/__init__.py")
# Make helper function available from 'from mypackage.utils import ...'
from .helpers import format_output 

__all__ = ['format_output']
```

```python
# mypackage/utils/helpers.py
"""Helper functions."""

def format_output(value, prefix=""):
    print(f"Helper formatting: {value}")
    return f"{prefix} {str(value).upper()}"
```

```python
# main_script.py (outside the package directory)
import logging
logging.basicConfig(level=logging.INFO)

print("--- Starting main_script.py ---")

# --- Various ways to import from the package --- 

# Import the whole package (executes mypackage/__init__.py)
print("\nImporting mypackage...")
import mypackage
print(f"Accessing package variable: {mypackage.PACKAGE_VERSION}")
# Access function made available in mypackage/__init__.py
result1 = mypackage.process_data("data1") 
print(f"Result 1: {result1}")
result_fmt = mypackage.format_output("hello")
print(f"Result fmt: {result_fmt}")

# Import a specific submodule
print("\nImporting mypackage.core_logic...")
import mypackage.core_logic
result2 = mypackage.core_logic.process_data("data2")
print(f"Result 2: {result2}")

# Import a submodule from a subpackage
print("\nImporting mypackage.utils.helpers...")
import mypackage.utils.helpers as hlp # Using alias
result3 = hlp.format_output("data3", prefix="[HLP]")
print(f"Result 3: {result3}")

# Import specific names using 'from'
print("\nImporting specific names using 'from'...")
# This works because __init__.py made them available
from mypackage import process_data, format_output 
result4 = process_data("data4")
result5 = format_output("data5")
print(f"Result 4: {result4}")
print(f"Result 5: {result5}")

# Import directly from a submodule
from mypackage.core_logic import process_data as core_process
result6 = core_process("data6")
print(f"Result 6: {result6}")

# Import from the subpackage utils (made available by its __init__.py)
from mypackage.utils import format_output as util_format
result7 = util_format("data7")
print(f"Result 7: {result7}")

# Test 'from mypackage import *' (relies on __all__ in mypackage/__init__.py)
print("\nTesting 'from mypackage import *'...")
# from mypackage import * 
# result_star = process_data("star") # Should work if __all__ is defined correctly
# print(f"Result star: {result_star}")
# print(f"core_logic available via star? {'core_logic' in locals()}")
print("(Star import example commented out - generally discouraged)")

print("\n--- Finished main_script.py ---")
```

**(Note:** To run the `main_script.py` example, you would need to create the specified directory structure and files.)

## 4. The Import System: Deeper Dive

### 4.1 Absolute vs. Relative Imports

*   **Absolute Imports:** Specify the full path from the project's root directory (or a directory in `sys.path`). Preferred for clarity and robustness, especially in applications and larger libraries.
    ```python
    import mypackage.core_logic
    from mypackage.utils import helpers
    from mypackage.utils.helpers import format_output
    ```
*   **Relative Imports:** Use dots (`.`) to indicate relative position *within the same package*. Useful for making packages more self-contained and easier to rename or move.
    *   `.`: Current directory (where the importing module resides).
    *   `..`: Parent directory.
    *   `...`: Grandparent directory, etc.
    ```python
    # Inside mypackage/core_logic.py:
    from . import utils           # Import utils subpackage from the same level
    from .utils import helpers   # Import helpers module from utils subpackage
    from .utils.helpers import format_output # Import specific name

    # Inside mypackage/utils/helpers.py:
    # from .. import core_logic   # Import core_logic from the parent package
    ```

**Key Rule:** Relative imports can **only** be used within modules that are part of a package. You **cannot** use them in top-level scripts that are run directly (files whose `__name__` is `'__main__'`). Trying to do so results in `ImportError: attempted relative import with no known parent package`.

**Best Practice:** Use absolute imports for clarity in most application code. Use relative imports *within* a package to refer to sibling or parent modules/subpackages, enhancing the package's internal cohesion.

### 4.2 Common Import Pitfalls

1.  **Circular Imports:** Occur when Module A imports Module B, and Module B imports Module A (directly or indirectly). Python's import mechanism can usually handle simple cases, but complex circular dependencies often lead to `ImportError` or `AttributeError` because one module might try to access a name from the other before it has been fully initialized.
    *   **Solution:** Refactor code. Move shared dependencies to a third module, import modules only within functions where needed (lazy import - less ideal), or rethink the overall structure.

2.  **Namespace Pollution:** Using `from module import *` dumps all names from the imported module into the current namespace, potentially overwriting existing names and making it hard to track the origin of functions/variables.
    *   **Solution:** Avoid star imports. Use `import module` or `from module import specific_name`.

3.  **Shadowing Built-ins or Standard Library Modules:** Naming your own module or variable the same as a Python built-in (e.g., `str.py`, `list.py`) or a standard library module (e.g., `math.py`, `email.py`) can prevent you from importing the original.
    *   **Solution:** Choose unique and descriptive names for your modules.

4.  **Side Effects in Modules:** Code at the top level of a module (outside functions/classes) runs *when the module is first imported*. Avoid complex operations or operations with significant side effects (like connecting to databases, modifying global state extensively) at the module's top level. Put such code in functions or classes.

5.  **Mutable Objects as Module Globals:** If a module defines a global variable that is a mutable object (like a list or dict), changes made to it by one importer will be seen by all other importers of that module, which can lead to unexpected behavior.
    *   **Solution:** Be cautious with mutable globals. Prefer passing state explicitly or managing it within classes.

## 5. Best Practices and Modern Concepts

1.  **PEP 8 Import Guidelines:**
    *   Imports should usually be on separate lines.
    *   Group imports in this order: Standard library, related third-party, local application/library specific.
    *   Use absolute imports generally.
    *   Avoid wildcard imports (`from ... import *`).
    *   Use tools like `isort` or formatters like `black` to automatically manage import order and formatting.

2.  **Using `__all__`:** Explicitly define `__all__` in your `__init__.py` files if you want to control what `from package import *` does, or simply as a way to document the public API of the package/module.

3.  **Namespace Packages (Python 3.3+):** Useful for large libraries or plugins that might be installed independently but contribute to the same top-level package namespace. They allow directories without `__init__.py` to be part of the package path. Less common for typical application structure.

4.  **Virtual Environments:** **Absolutely essential.** Always use virtual environments (`venv`, `conda`) to isolate project dependencies. This prevents conflicts between different projects requiring different versions of the same library.
    ```bash
    # Create a virtual environment (using venv)
    python -m venv .venv 
    # Activate it (Linux/macOS)
    source .venv/bin/activate
    # Activate it (Windows - Git Bash/WSL)
    # source .venv/Scripts/activate
    # Activate it (Windows - Cmd/PowerShell)
    # .venv\Scripts\activate.bat 
    # .venv\Scripts\Activate.ps1
    
    # Install packages (they go into the venv)
    pip install requests
    
    # Deactivate when done
    deactivate
    ```

5.  **Editable Installs (`pip install -e .`):** During development of a package, install it in 'editable' mode. This creates links in `site-packages` pointing back to your source code, so changes you make are immediately reflected without needing to reinstall after every edit.
    ```bash
    # In your project's root directory (containing pyproject.toml or setup.py)
    # Ensure your virtual environment is active
    pip install -e .
    ```

6.  **Packaging and Distribution (`pyproject.toml`):** For sharing your package or installing it cleanly, define its metadata, dependencies, and build system in a `pyproject.toml` file (modern standard replacing `setup.py` for most configurations). Use tools like `pip`, `build`, `twine`, or higher-level managers like `Poetry` or `Hatch`.

7.  **Type Hinting:** Use type hints within your modules and packages to improve code clarity, enable static analysis (with tools like `mypy`), and enhance editor support.

## 6. Enterprise Considerations

*   **Project Structure:** Larger applications often adopt standardized structures (e.g., separating tests, configuration, application code, documentation). Frameworks like Django or Flask often impose a structure.
    ```
    my_enterprise_app/
    ├── .venv/
    ├── src/                  # Main source code often here
    │   └── myapp/            # Your main package
    │       ├── __init__.py
    │       ├── api/
    │       ├── core/
    │       ├── models/
    │       └── utils/
    ├── tests/                # Unit and integration tests
    │   ├── __init__.py
    │   ├── test_api.py
    │   └── test_core.py
    ├── data/                 # Data files (if needed)
    ├── docs/                 # Documentation
    ├── scripts/              # Utility scripts
    ├── config/               # Configuration files
    ├── .gitignore
    ├── pyproject.toml        # Project metadata and dependencies
    ├── README.md
    └── ...
    ```
*   **Dependency Management:** Explicitly declare all project dependencies (`requirements.txt` or preferably `pyproject.toml`). Use tools like `pip-tools` or Poetry/Hatch to manage pinned versions for reproducible builds.
*   **Testing:** Write tests (`unittest`, `pytest`) for your modules and packages to ensure correctness and prevent regressions. Test import behavior and package structure.
*   **CI/CD:** Integrate automated testing, linting, and packaging into your Continuous Integration / Continuous Deployment pipelines.
*   **Code Reviews:** Review import strategies, module coupling, and package structure during code reviews.

## 7. Pitfalls Revisited & Interview Questions

**Key Pitfalls Summary:**
*   Circular imports.
*   Wildcard imports (`from ... import *`).
*   Shadowing names.
*   Side effects at module level.
*   Relative imports outside packages.
*   Not using virtual environments.

**Common Interview Questions:**

1.  What is the difference between a module and a package in Python?
2.  What is the purpose of the `__init__.py` file?
3.  How does Python find modules when you import them? (Explain `sys.path`).
4.  Explain the difference between `import mymodule` and `from mymodule import myfunction`.
5.  What is an import alias, and why might you use one?
6.  What is a circular import, and how can you resolve it?
7.  Why is `from module import *` generally discouraged?
8.  Explain the difference between absolute and relative imports. When would you use each?
9.  What does the `if __name__ == '__main__':` block do?
10. What is a virtual environment, and why is it important?
11. What is the role of `pyproject.toml` in modern Python packaging?
12. (Advanced) What is a namespace package?

## 8. Challenge: Create a Simple Calculator Package

**Goal:** Build a basic calculator package with separate modules for operations and history.

**Tasks:**

1.  **Create Package Structure:**
    ```
    calculator/
    ├── __init__.py
    ├── operations.py
    └── history.py 
    ```
2.  **Implement `operations.py`:**
    *   Define functions `add(a, b)`, `subtract(a, b)`, `multiply(a, b)`, `divide(a, b)`.
    *   The `divide` function should handle `ZeroDivisionError` gracefully (e.g., return `float('inf')` or `None` and log a warning).
3.  **Implement `history.py`:**
    *   Define a global list (e.g., `_calculation_history = []`) to store calculation records (be mindful of mutable globals - perhaps better in a class, but a list is okay for this simple challenge).
    *   Define a function `add_to_history(operation: str, a, b, result)` that appends a formatted string or tuple representing the calculation to the `_calculation_history` list.
    *   Define a function `get_history()` that returns a copy of the history list.
    *   Define a function `clear_history()`.
4.  **Implement `calculator/__init__.py`:**
    *   Import the core operation functions (`add`, `subtract`, `multiply`, `divide`) from `.operations`.
    *   Import the history functions (`get_history`, `clear_history`) from `.history`.
    *   Use `__all__` to define the public API of the `calculator` package (e.g., `['add', 'subtract', 'multiply', 'divide', 'get_history', 'clear_history']`).
5.  **Write Test Script (`test_calculator.py` - outside the package):**
    *   Import the `calculator` package.
    *   Perform several calculations using the imported functions (e.g., `calculator.add(5, 3)`, `calculator.divide(10, 2)`, `calculator.divide(5, 0)`).
    *   **Modify `operations.py`:** Make sure the operation functions call `history.add_to_history` (using a relative import like `from . import history`) *before* returning the result.
    *   Print the calculation history using `calculator.get_history()`.
    *   Clear the history and verify it's empty.

**(Bonus):** Refactor `history.py` to use a class `CalculationHistory` to manage the history list instead of using a global variable.

In [7]:
# --- Solution Setup (Illustrative - Create files manually) ---

# --- calculator/__init__.py ---
# """Basic Calculator Package"""
# print("Initializing Calculator Package...")
# from .operations import add, subtract, multiply, divide
# from .history import get_history, clear_history, add_to_history # Expose add_to_history temporarily if needed directly
# 
# __all__ = ['add', 'subtract', 'multiply', 'divide', 'get_history', 'clear_history']

# --- calculator/history.py ---
# """Calculation History Module"""
# import logging
# _calculation_history = []
# logger = logging.getLogger(__name__)
# 
# def add_to_history(operation: str, a, b, result):
#     record = f"{a} {operation} {b} = {result}"
#     _calculation_history.append(record)
#     logger.debug(f"Added to history: {record}")
# 
# def get_history():
#     return _calculation_history.copy()
# 
# def clear_history():
#     _calculation_history.clear()
#     logger.info("Calculation history cleared.")

# --- calculator/operations.py ---
# """Core Arithmetic Operations"""
# import logging
# from . import history # Relative import
# logger = logging.getLogger(__name__)
# 
# def add(a, b):
#     result = a + b
#     history.add_to_history('+', a, b, result)
#     return result
# 
# def subtract(a, b):
#     result = a - b
#     history.add_to_history('-', a, b, result)
#     return result
# 
# def multiply(a, b):
#     result = a * b
#     history.add_to_history('*', a, b, result)
#     return result
# 
# def divide(a, b):
#     try:
#         result = a / b
#         history.add_to_history('/', a, b, result)
#         return result
#     except ZeroDivisionError:
#         logger.warning(f"Attempted division by zero: {a}/{b}")
#         result = float('inf') # Or None, or raise custom error
#         history.add_to_history('/', a, b, 'Error: Division by Zero')
#         return result 

# --- test_calculator.py (Run this file after creating the package) ---
import logging
logging.basicConfig(level=logging.INFO)

# Assuming 'calculator' directory is in the same parent directory
# or the parent directory is in sys.path
try:
    import calculator
    
    print("--- Testing Calculator --- ")
    r1 = calculator.add(10, 5)
    print(f"10 + 5 = {r1}")
    
    r2 = calculator.subtract(10, 5)
    print(f"10 - 5 = {r2}")
    
    r3 = calculator.multiply(10, 5)
    print(f"10 * 5 = {r3}")
    
    r4 = calculator.divide(10, 5)
    print(f"10 / 5 = {r4}")
    
    r5 = calculator.divide(10, 0)
    print(f"10 / 0 = {r5}")
    
    print("\n--- Calculation History ---")
    history_list = calculator.get_history()
    for item in history_list:
        print(f"  {item}")
        
    calculator.clear_history()
    print(f"\nHistory after clear: {calculator.get_history()}")

except ImportError as e:
    print(f"\nERROR: Could not import the 'calculator' package. {e}")
    print("Ensure the 'calculator' directory with __init__.py exists relative to this script or in sys.path.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")



ERROR: Could not import the 'calculator' package. No module named 'calculator'
Ensure the 'calculator' directory with __init__.py exists relative to this script or in sys.path.


## 9. Conclusion

Modules and packages are the foundation upon which well-structured, maintainable, and scalable Python applications are built. By understanding how to create modules, organize them into packages using `__init__.py`, master Python's import system (including absolute and relative imports), and adhere to best practices like using virtual environments and clear project structure, you can effectively manage code complexity.

As you progress, continue exploring Python's packaging ecosystem (`pyproject.toml`, build tools) to share your code or manage dependencies in larger projects effectively.